In [ ]:
from datamarket import *
from random import sample, choice
from sys import maxsize
import string
from sklearn.linear_model import LinearRegression
import time
import io
import cProfile, pstats
import pickle as pkl
import csv

In [ ]:
dimensions = 4
dimensions_size = 100

In [ ]:
# 1st dimension is A, 2nd dimension is B, ...
def get_dimension_name(d):
    if d < 26:
      return string.ascii_uppercase[d]
    else:
      res = ''
      for i in range(0, d, 26):
        res += string.ascii_uppercase[d % 26]
      return res

def generate_dimension_table(d):
    name = get_dimension_name(d)
    df = pd.DataFrame()
    df[name] = range(dimensions_size)
    df[name + "_att"] = choices(range(50), k=dimensions_size)
    return df

In [ ]:
#okay, let's design a little...
#1. for each buyer ds, generate 2 or 3 join keys, that's it.
#2. for each seller ds corresponding to a buyer ds, generate a unique key column,
#randomly choose one of the buyer join keys to be in the seller dataset, add it to the seller dataset,
#and then add an _att column we'll use to synthesize a result for a few seller tables, like before
#3. replicate each buyer and generate a bad buyer, who can join with the same sellers,
#but whose result is a different linear combination
def gen_buyers_and_sellers(buyers, sellers, num_jks, pred_features):
  #first, generate buyer tables, which are fact tables with just a few key columns.
  total_buyers = []
  total_bad_buyers = []
  total_sellers = []
  b_sellermap = {}
  ground_truth = {}
  d_cnt = 0
  s_cnt = 0

  for b in range(buyers):
    ground_truth[b] = []
    b_sellers = []
    #generate buyer tables with 2 keys
    b_table = pd.DataFrame()
    for j in range(num_jks):
      name = get_dimension_name(d_cnt)
      d_cnt += 1
      b_table[name] = range(dimensions_size)
    
    #now, generate sellers who can join with this buyer
    for s in range(sellers):
      s_tbl = pd.DataFrame()
      #add one of the join keys to seller
      sjk = choice(b_table.columns)
      s_tbl[sjk] = b_table[sjk]
      #for c in b_table.columns:
      #  s_tbl[c] = b_table[c]
      s_tbl['seller' + str(s_cnt) + '_att'] = choices(range(50), k=dimensions_size)
      b_sellers.append(s_tbl)
      s_cnt += 1
    
    total_sellers += b_sellers
    b_sellermap[b] = b_sellers
    
    #now, generate the result column for the buyer
    pred_sellers = sample(range(sellers), pred_features)
    b_table['result'] = 0
    for ps in pred_sellers:
      sdf = b_sellers[ps]
      predf = [c for c in sdf.columns if '_att' in c][0] #there should only be one such att
      c = choice(range(1,10))
      b_table['result'] += c * sdf[predf]
      print("Pred col and Constant: {}, {}".format(predf, c))
      ground_truth[b].append(predf) #now, we can only tell the seller from its exact attribute, and not its key
    total_buyers.append(b_table)
  
  #now, generate bad copies...
  for i,bdf in enumerate(total_buyers):
    bad_bdf = bdf.copy()
    b_sel = b_sellermap[i]
    ground_truth[buyers + i] = []
    #now, generate a new result column for the buyer
    pred_sellers = sample(range(sellers), pred_features)
    bad_bdf['result'] = 0
    for ps in pred_sellers:
      sdf = b_sel[ps]
      predf = [c for c in sdf.columns if '_att' in c][0] #there should only be one such att
      c = choice(range(1,10))
      bad_bdf['result'] += c * sdf[predf]
      print("Bad Buyer Pred col and Constant: {}, {}".format(predf, c))
      ground_truth[buyers + i].append(predf) #now, we can only tell the seller from its exact attribute, and not its key
    total_bad_buyers.append(bad_bdf)

  return total_buyers, total_bad_buyers, total_sellers, ground_truth


In [ ]:
buyerdfs, badbuyers, sellerdfs, ground_truth = gen_buyers_and_sellers(10, 300, 2, 2)

Pred col and Constant: seller252_att, 5
Pred col and Constant: seller80_att, 2
Pred col and Constant: seller576_att, 9
Pred col and Constant: seller511_att, 9
Pred col and Constant: seller626_att, 3
Pred col and Constant: seller824_att, 8
Pred col and Constant: seller1185_att, 3
Pred col and Constant: seller1057_att, 3
Pred col and Constant: seller1225_att, 1
Pred col and Constant: seller1363_att, 8
Pred col and Constant: seller1743_att, 9
Pred col and Constant: seller1751_att, 8
Pred col and Constant: seller1925_att, 7
Pred col and Constant: seller2086_att, 1
Pred col and Constant: seller2362_att, 2
Pred col and Constant: seller2106_att, 9
Pred col and Constant: seller2491_att, 2
Pred col and Constant: seller2559_att, 1
Pred col and Constant: seller2723_att, 1
Pred col and Constant: seller2839_att, 7
Bad Buyer Pred col and Constant: seller287_att, 1
Bad Buyer Pred col and Constant: seller243_att, 5
Bad Buyer Pred col and Constant: seller549_att, 8
Bad Buyer Pred col and Constant: sell

In [ ]:
ground_truth

{0: ['seller252_att', 'seller80_att'],
 1: ['seller576_att', 'seller511_att'],
 2: ['seller626_att', 'seller824_att'],
 3: ['seller1185_att', 'seller1057_att'],
 4: ['seller1225_att', 'seller1363_att'],
 5: ['seller1743_att', 'seller1751_att'],
 6: ['seller1925_att', 'seller2086_att'],
 7: ['seller2362_att', 'seller2106_att'],
 8: ['seller2491_att', 'seller2559_att'],
 9: ['seller2723_att', 'seller2839_att'],
 10: ['seller287_att', 'seller243_att'],
 11: ['seller549_att', 'seller303_att'],
 12: ['seller776_att', 'seller634_att'],
 13: ['seller1089_att', 'seller1048_att'],
 14: ['seller1475_att', 'seller1309_att'],
 15: ['seller1775_att', 'seller1557_att'],
 16: ['seller2000_att', 'seller1822_att'],
 17: ['seller2339_att', 'seller2173_att'],
 18: ['seller2688_att', 'seller2585_att'],
 19: ['seller2990_att', 'seller2806_att']}

In [ ]:
buyerdfs[0]

,A,B,result
0,0,0,117
1,1,1,71
2,2,2,26
3,3,3,75
4,4,4,59
...,...,...,...
95,95,95,163
96,96,96,183
97,97,97,196
98,98,98,324


In [ ]:
badbuyers[0]

,A,B,result
0,0,0,182
1,1,1,275
2,2,2,289
3,3,3,137
4,4,4,234
...,...,...,...
95,95,95,300
96,96,96,274
97,97,97,318
98,98,98,56


In [ ]:
[s for s in sellerdfs if 'seller90_att' in s.columns]

[     B  seller90_att
 0    0            39
 1    1            24
 2    2            14
 3    3            30
 4    4            39
 ..  ..           ...
 95  95            45
 96  96            45
 97  97            20
 98  98            36
 99  99            49
 
 [100 rows x 2 columns]]

In [ ]:
len(sellerdfs)

3000

In [ ]:
def comp_zipf(ind : int, alpha, n):
  denom = ind ** alpha
  denom *= sum([(1 / (i+1)) ** alpha for i in range(n)])
  res = 1.0 / denom
  return res

In [ ]:
def gen_bad_buyer(bdf):
  #this needs a redesign...we don't necessarily want that replicated buyer tables can't perform well on any dataset in the market,
  #rather, we want that the replica performs well on some other seller datasets, not the same one as the buyer.
  res = bdf.copy()
  res['result'] = choices(range(50000), k=len(res['result']))
  return res

In [ ]:
#next, generate a sequence based on the zipf distribution, and based on a probability of goodness for similar buyers
#generate a sequence of potentially repeating buyers
def gen_buyer_seq(bdfs, bad_dfs, gt, seq_len, p_good, alpha, rand=False):
  #stagger buyers
  tot_buyers = []
  for i in range(len(bdfs)):
    tot_buyers.append(bdfs[i])
    tot_buyers.append(bad_dfs[i])
  
  seq_ground_truth = {}
  res_seq = []
  if rand:
    #assume ranking in ascending order, and compute probabilities per list index
    n = len(tot_buyers)
    zipfs = [comp_zipf(i+1, alpha, n) for i in range(n)]
    #now, randomly choose seq_len dataframes from the input list
    #and add the corresponding indices to the ground truth
    pick_inds = choices(range(n), weights=zipfs, k=seq_len)
    print(pick_inds)
    for i,p in enumerate(pick_inds):
      #print(i, p)
      seq_ground_truth[i] = gt[p]
      #flip a coin to decide whether copy will be good or bad
      good = choices([True,False], weights=[p_good, 1 - p_good], k=1)[0]
      #print(good)
      #good = False
      res_seq.append(tot_buyers[p])
    return res_seq, seq_ground_truth
  else:
    return bdfs, gt
rand_b_seq, ground_truth = gen_buyer_seq(buyerdfs, badbuyers, ground_truth, 50, 0.5, 4, rand=True)

[1, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
rand_b_seq[0]

,A,B,result
0,0,0,182
1,1,1,275
2,2,2,289
3,3,3,137
4,4,4,234
...,...,...,...
95,95,95,300
96,96,96,274
97,97,97,318
98,98,98,56


In [ ]:
rand_b_seq[1]

,A,B,result
0,0,0,117
1,1,1,71
2,2,2,26
3,3,3,75
4,4,4,59
...,...,...,...
95,95,95,163
96,96,96,183
97,97,97,196
98,98,98,324


In [ ]:
ground_truth

{0: ['seller106_att', 'seller13_att'],
 1: ['seller106_att', 'seller13_att'],
 2: ['seller106_att', 'seller13_att'],
 3: ['seller106_att', 'seller13_att'],
 4: ['seller106_att', 'seller13_att'],
 5: ['seller106_att', 'seller13_att'],
 6: ['seller106_att', 'seller13_att'],
 7: ['seller106_att', 'seller13_att'],
 8: ['seller106_att', 'seller13_att'],
 9: ['seller106_att', 'seller13_att'],
 10: ['seller106_att', 'seller13_att'],
 11: ['seller106_att', 'seller13_att'],
 12: ['seller106_att', 'seller13_att'],
 13: ['seller106_att', 'seller13_att'],
 14: ['seller106_att', 'seller13_att'],
 15: ['seller106_att', 'seller13_att'],
 16: ['seller106_att', 'seller13_att'],
 17: ['seller106_att', 'seller13_att'],
 18: ['seller106_att', 'seller13_att'],
 19: ['seller106_att', 'seller13_att'],
 20: ['seller106_att', 'seller13_att'],
 21: ['seller106_att', 'seller13_att'],
 22: ['seller106_att', 'seller13_att'],
 23: ['seller106_att', 'seller13_att'],
 24: ['seller106_att', 'seller13_att'],
 25: ['sel

In [ ]:
#first, test to see if what we generate is actually predictive
#print(ground_truth)
b0df = rand_b_seq[0]
#print(b0df.columns)
s0dfs = [sdf for sdf in sellerdfs if ground_truth[0][0] in sdf.columns or ground_truth[0][1] in sdf.columns]
#print(s0dfs)
join1df = pd.merge(b0df, s0dfs[0])
join2df = pd.merge(join1df, s0dfs[1])
print(join2df)

     A   B  result  seller95_att  seller265_att
0    0   0     182            22             24
1    1   1     275            37             30
2    2   2     289            32             43
3    3   3     137             7             34
4    4   4     234            33             23
..  ..  ..     ...           ...            ...
95  95  95     300            48             20
96  96  96     274            26             48
97  97  97     318            42             36
98  98  98      56            10              2
99  99  99     189            30             13

[100 rows x 5 columns]


In [ ]:
msk = split_mask(len(join2df)) < 0.8
join_train = join2df[msk].copy()
join_test = join2df[~msk].copy()

#numerics = ['float16', 'float32', 'float64']
#excols = ['Residents Total Confirmed COVID-19']
#excols += [c for c in b_train.columns if 'COVID' in c]
incols = [c for c in join_train.columns if '_att' in c]
#incols = [c for c in join_train.columns if 'B_att' == c]

X_b = join_train.loc[:, join_train.columns.isin(incols)]
y_b = join_train['result']
print(X_b)
print(y_b)

X_btest = join_test.loc[:, join_test.columns.isin(incols)]
y_btest = join_test['result']
print(X_btest)
print(y_btest)


reg = LinearRegression().fit(X_b, y_b)
print('manual join on 0: ' + str(reg.score(X_btest, y_btest)))

    seller95_att  seller265_att
0             22             24
1             37             30
2             32             43
3              7             34
4             33             23
..           ...            ...
94            13             49
95            48             20
96            26             48
97            42             36
99            30             13

[83 rows x 2 columns]
0     182
1     275
2     289
3     137
4     234
     ... 
94    212
95    300
96    274
97    318
99    189
Name: result, Length: 83, dtype: int64
    seller95_att  seller265_att
12            19             45
13            33              9
15            34             47
18            34              3
19            46             24
22             9             34
23            22             34
24            14             26
25            27             16
30             5             16
41            25              8
54            35             25
68             8            

In [ ]:
'seller0_att'[:-4]

'seller0'

In [ ]:
#now, I have all the buyer and seller dataframes.
#add all the seller dataframes to the market, 
sellers = []
for s in sellerdfs:
  s_data = agg_dataset()
  s_jks = [c for c in s.columns if '_att' not in c]
  s_name = [c for c in s.columns if '_att' in c][0][:-4]
  s_atts = [c for c in s.columns if '_att' in c]
  s_data.load(s, [], s_jks, s_name)
  s_data.find_features()
  s_data.remove_redundant_columns()
  s_data.create_count_true()
  s_data.compute_agg(True)
  sellers.append((s_data, s_jks[0])) #every seller only has one join key for us

In [ ]:
sellers[0][0].data

,B,cov:s:seller0:seller0_att,cov:c,"cov:Q:seller0:seller0_att,seller0:seller0_att"
0,0,15,1,225
1,1,19,1,361
2,2,37,1,1369
3,3,27,1,729
4,4,32,1,1024
...,...,...,...,...
95,95,13,1,169
96,96,10,1,100
97,97,40,1,1600
98,98,18,1,324


In [ ]:
#define the cache...
class RequestCache:
    def __init__(self, cache_size):
      self.cache_size = cache_size
      self.cnt = 0
      self.cdict = {}
  
    def find_lfu(self):
      min_freq = maxsize
      cur_sc = None
      for sc in self.cdict:
        aug_plan, freq_cnt = self.cdict[sc]
        if freq_cnt < min_freq:
          cur_sc = sc
          min_freq = freq_cnt
      
      return (cur_sc, min_freq)
    

    def read_el(self, schema : tuple):
      if schema in self.cdict:
        aug_plan, freq_cnt = self.cdict[schema]
        #update freq count
        self.cdict[schema] = (aug_plan, freq_cnt + 1)
        return aug_plan
      else:
        #now, we don't have a match.
        #if we have more items than entries in the dictionary...
        if self.cnt >= self.cache_size:
          #evict LFU
          min_sc, f_cnt = self.find_lfu()
          del self.cdict[min_sc]
          self.cnt -= 1
        #add schema and aug plan...is what we'd like to do, but we don't have the aug plan when we check this.
        #so, return None at this point.
        return None
    
    #...and then, later, when we have the aug plan, we can add it.
    #the aug plan is specified as a dictionary, where key is the join key to a seller table,
    #and the value is the set of attributes to use
    def add_el(self, schema : tuple, aug_plan : list):
      self.cdict[schema] = (aug_plan, 1)
      self.cnt += 1
      
      


In [ ]:
def find_seller_from_dim(indim):
  for sellerdata, dimension in sellers:
    if dimension == indim:
      seller_attr = [d for d in sellerdata.agg_dimensions if '_att' in d]
      return sellerdata, dimension, seller_attr

In [ ]:
def find_optimal_plan(buyer, ind, gt):
  #we also need to get the correct timing--exclude loading the buyers,
  #because this was supposed to be a preprocessing step
  msk = split_mask(len(buyer)) < 0.8
  b_train = buyer[msk].copy()
  b_test = buyer[~msk].copy()

  b_train_data = agg_dataset()
  b_keys = [b for b in buyer.columns.tolist() if b != 'result']
  b_train_data.load(b_train, ['result'], b_keys, "buyer" + str(ind))
  b_train_data.process_target("result")
  b_train_data.to_numeric_and_impute_all()
  b_train_data.remove_redundant_columns()
  b_train_data.create_count_true()
  b_train_data.compute_agg()

  b_test_data = agg_dataset()
  b_test_data.load(b_test, ['result'], b_keys, "buyer" + str(ind))
  b_test_data.process_target("result")
  b_test_data.to_numeric_and_impute_all()
  b_test_data.remove_redundant_columns()
  b_test_data.create_count_true()
  b_test_data.compute_agg()

  m = 2
  y = "result"
  aug_plan = []
  #now, do augmentation for the buyer...
  a_st = time.time()
  for i in range(m):
      best_seller = None
      best_seller_attrs = []
      best_dimension = None
      best_seller_ind = -(maxsize - 1) #it should be really obvious if nothing gets chosen...
      best_r2 = -(maxsize - 1)
      dim_cnt = 0
      for j, sd in enumerate(sellers):
          sellerdata = sd[0]
          dimension = sd[1]
          #print("seller tried: {}".format(sellerdata.name))
          #print("dimension tried: {}".format(dimension))
          # check if current seller has been bought
          if sellerdata.name in b_train_data.datasets:
              continue
          #also check if the current seller can even be joined with the buyer
          #if not, skip it
          if dimension not in b_train_data.agg_dimensions and dimension not in b_test_data.agg_dimensions:
            continue

          # find the attributes and r2 of augmenting
          #print("selecting features")
          #print("dimension tried: {}".format(dimension))
          dim_cnt += 1
          cur_atts, final_r2 = select_features(b_train_data, b_test_data, sellerdata, dimension, 4, y)
  #         cur_atts, final_r2 = select_features(gender_train, gender_test, sellerdata, dimension,10)

          if final_r2 > best_r2:
              best_seller = sellerdata
              best_dimension = dimension
              best_seller_attrs = cur_atts
              best_seller_ind = j
              best_r2 = final_r2
      
      #if our results were garbage, replace them with what we know is best
      #if best_r2 <= 10 ** (-9) and ind in gt: #then, it's essentially 0 or worse
      #    print("Added Ground Truth")
      #    best_seller, best_dimension, best_seller_attrs = find_seller_from_dim(gt[ind][i])
      #    print("best_seller.X: {}".format(best_seller.X))
      #    best_seller_attrs = best_seller.X
      #    _, best_r2 = select_features(b_train_data, b_test_data, best_seller, best_dimension, 4, y)

      #print("Buyer schema: {}".format(b_train.columns))
      print(best_seller.name, best_seller_attrs, best_r2)
      aug_plan.append(best_seller_ind)
      true_absorb = True
      budget = 100000
      
      # there is no attributes that is predictive in seller dataset
      if len([x for x in best_seller_attrs if x in best_seller.X]) == 0:
          true_absorb = False
      else:
          count_true_join = connect_count(b_train_data, best_seller, best_dimension)
          if count_true_join["cov:c"].sum() > budget:
              print("beyond budget!!")
              true_absorb = False
          else:
              b_train_data.count_true = count_true_join
              b_train_data.count_true.reset_index(inplace=True)
      print(true_absorb)
      if not true_absorb:
          b_train_data.datasets.add(best_seller.name)
          b_test_data.datasets.add(best_seller.name)
          #we just want to test the cache, we don't really care about R2
          #gender_train_data.absorb(best_seller, best_dimension, best_seller_attrs + [gender_train_data.name + ":" + y])
          #gender_test_data.absorb(best_seller, best_dimension, best_seller_attrs + [gender_train_data.name + ":" + y])
          #cache_augs["gender"].append((best_seller, best_dimension, best_seller_attrs))
      else:
          b_train_data.absorb(best_seller, best_dimension, best_seller_attrs + [b_train_data.name + ":" + y])
          #print("Absorbed Training data")
          #print("New buyer pred atts:")
          #print(b_train_data.X)
          b_test_data.absorb(best_seller, best_dimension, best_seller_attrs + [b_train_data.name + ":" + y])
          #print("Absorbed Testing data")
      print("Number of dimensions tried: {}".format(dim_cnt))
      #cache_augs["gender"].append((best_seller, best_dimension, best_seller_attrs))
  a_en = time.time()
  aug_time = a_en - a_st
  return aug_plan, aug_time

In [ ]:
#%%time
def cache_optimal_plan(buyer, ind, gt, cache_size, cache_state=None):
    #we need to get the cache time separately here, to avoid buyer loading
    #under all circumstances...
    cache_hit = False
    used_aug = False
    tot_coptime = 0.0
    ca_st = time.time()
    #initialize the cache...
    if cache_state == None:
        cache = RequestCache(cache_size)
    else:
        cache = cache_state
        print(cache_state)
    
    buyerdf = buyer
    b_schema = tuple(buyerdf.columns.tolist())
    #print(b_schema)
    #print(cache.cdict.keys())
    aug_plan = cache.read_el(b_schema)
    ca_en = time.time()
    tot_coptime += ca_en - ca_st
    #print("Cache count before: {}".format(cache.cnt))
    if aug_plan != None:
        cache_hit = True
        used_aug = True
        #print("Number of augmentations to make: {}".format(len(aug_plan)))
        best_r2 = -(maxsize - 1)
        use_aug = True
        for s in aug_plan:
            #actually, in this case, there are many sellers with the same join key
            #but each seller has a distinct name for attributes, so we use that
            #but sellers doesn't change while we run all this! So, it's much
            #easier to just assume we have a unique way of identifying which seller
            #tables should be used from join keys, just like in a real database
            #and just store the seller indices in the list.
            #print("Using augmentation plan")
            s_match = sellers[s][0]
            jk = sellers[s][1]

            b_start = time.time()
            msk = split_mask(len(buyer)) < 0.8
            b_train = buyer[msk].copy()
            b_test = buyer[~msk].copy()

            b_train_data = agg_dataset()
            b_keys = [b for b in buyer.columns.tolist() if b != 'result']
            b_train_data.load(b_train, ['result'], b_keys, "buyer" + str(ind))
            b_train_data.process_target("result")
            b_train_data.to_numeric_and_impute_all()
            b_train_data.remove_redundant_columns()
            b_train_data.create_count_true()
            b_train_data.compute_agg()

            b_test_data = agg_dataset()
            b_test_data.load(b_test, ['result'], b_keys, "buyer" + str(ind))
            b_test_data.process_target("result")
            b_test_data.to_numeric_and_impute_all()
            b_test_data.remove_redundant_columns()
            b_test_data.create_count_true()
            b_test_data.compute_agg()
            b_end = time.time()
            print("Buyer init: {}".format(b_end - b_start))

            y = 'result'
            
            #print("s_match cols: {}".format(s_match.columns))
            # find the attributes and r2 of augmenting
            sf_st = time.time()
            cur_atts, final_r2 = select_features(b_train_data, b_test_data, s_match, jk, 4, y)
            sf_en = time.time()
            print("SF time: {}".format(sf_en - sf_st))
            tot_coptime += sf_en - sf_st
      
            #this is a check to see if our r2 is really improving.
            #if we find out it's not, then this augmentation plan is no good, and we need to move to 
            if final_r2 > best_r2:
              best_r2 = final_r2
            
            ab_st = time.time()
            b_train_data.absorb(s_match, jk, cur_atts + [b_train_data.name + ":" + y])
            b_test_data.absorb(s_match, jk, cur_atts + [b_train_data.name + ":" + y])
            ab_en = time.time()
            print("Abs time: {}".format(ab_en - ab_st))
            tot_coptime += ab_en - ab_st
        
        #once we've executed the plan, check the r2

        if best_r2 <= 0:
            use_aug = False
    
        if not use_aug:
            used_aug = False
            print("Augmentation plan failed--finding optimal plan manually")
            #raise Exception("Augmentation plan failed--finding optimal plan manually")

            opt_aug, opt_time = find_optimal_plan(buyer, ind, gt)
            #print("Cache count just before: {}".format(cache.cnt))
            cadd_st = time.time()
            cache.add_el(b_schema, opt_aug)
            cadd_en = time.time()
            tot_coptime += cadd_en - cadd_st + opt_time
            #print("New keys: {}".format(cache.cdict.keys()))
            #print("Cache count immediately after: {}".format(cache.cnt))
    else:
        cache_hit = False
        print("No existing plan found")
        opt_aug, opt_time = find_optimal_plan(buyer, ind, gt)
        cadd_st = time.time()
        cache.add_el(b_schema, opt_aug)
        cadd_en = time.time()
        tot_coptime += cadd_en - cadd_st + opt_time
        #print("New keys: {}".format(cache.cdict.keys()))
        #print("Cache count first time after: {}".format(cache.cnt))
    
    return cache, tot_coptime, cache_hit, used_aug

In [ ]:
len(sellers)

3000

In [ ]:
#first, create the csv file we'll want to write to
fh = open('zipf4_cachetimesv2.csv', 'w+')
fieldnames = ['latency', 'cache_hit', 'use_aug']
csvwriter = csv.DictWriter(fh, fieldnames)
csvwriter.writeheader()

In [ ]:
tot_ctime = 0.0
for i,b in enumerate(rand_b_seq):
  print("Looking at buyer {}".format(i))
  if i == 0:
        #c_start = time.time()
        new_state, new_time, was_hit, used_aug = cache_optimal_plan(b, i, ground_truth, 5)
        wh_int = -1
        if was_hit:
          wh_int = 1
        else:
          wh_int = 0
        ua_int = -1
        if used_aug:
          ua_int = 1
        else:
          ua_int = 0
        new_row = {'latency' : new_time, 'cache_hit' : wh_int, 'use_aug' : ua_int}
        csvwriter.writerow(new_row)
        #c_end = time.time()
        print("Before-cache runtime: {}".format(new_time))
        tot_ctime += new_time
        pkl.dump(new_state, open("cache_state.pkl", "wb+"))
  else:
      #we read the file in here so we don't hav
      with open('cache_state.pkl', 'rb') as f:
          cache_state = pkl.load(f)
      #c_start = time.time()
      #print("Cache size: {}".format(cache_state.cache_size))
      #pr = cProfile.Profile()
      #pr.enable()
      new_state, new_time, was_hit, used_aug = cache_optimal_plan(b, i, ground_truth, cache_state.cache_size, cache_state)
      #pr.disable()
      #s = io.StringIO()
      #ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
      #ps.print_stats()
      #with open('buyer' + str(i) + '_cache_test.txt', 'w+') as f:
      #      f.write(s.getvalue())
      #c_end = time.time()
      print("Cached runtime: {}".format(new_time))
      tot_ctime += new_time
      #write the new state
      pkl.dump(new_state, open("cache_state.pkl", "wb+"))
      #next, write the per-request results to csv file
      wh_int = -1
      if was_hit:
        wh_int = 1
      else:
        wh_int = 0
      ua_int = -1
      if used_aug:
        ua_int = 1
      else:
        ua_int = 0
      new_row = {'latency' : new_time, 'cache_hit' : wh_int, 'use_aug' : ua_int}
      csvwriter.writerow(new_row)
  print(new_state.cdict)
  #if i > 10:
  #  break
print("cache synth runtime: {}".format(tot_ctime))
fh.close()

Looking at buyer 0
No existing plan found


/content/gdrive/My Drive/Win22/MLforFineTuning/factorizedexperiments/datamarket.py:384: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  join.drop('cov:c_y', 1, inplace=True)


seller243 ['seller243:seller243_att'] 0.9039361689826442
True
Number of dimensions tried: 300


/content/gdrive/My Drive/Win22/MLforFineTuning/factorizedexperiments/datamarket.py:329: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  join.drop('cov:c_x', 1, inplace=True)
/content/gdrive/My Drive/Win22/MLforFineTuning/factorizedexperiments/datamarket.py:330: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  join.drop('cov:c_y', 1, inplace=True)


seller287 ['seller243:seller243_att', 'seller287:seller287_att'] 1.0000000000000029
True
Number of dimensions tried: 299
Before-cache runtime: 6.28612494468689
{('A', 'B', 'result'): ([243, 287], 1)}
Looking at buyer 1
Buyer init: 0.02801346778869629
SF time: 0.013488054275512695
Abs time: 0.021839380264282227
Buyer init: 0.028276443481445312
SF time: 0.013260602951049805
Abs time: 0.0215909481048584
Augmentation plan failed--finding optimal plan manually
seller252 ['seller252:seller252_att'] 0.8081640089508115
True
Number of dimensions tried: 300
seller80 ['seller252:seller252_att', 'seller80:seller80_att'] 0.9999999999999981
True
Number of dimensions tried: 299
Cached runtime: 6.334828615188599
{('A', 'B', 'result'): ([252, 80], 1)}
Looking at buyer 2
Buyer init: 0.023465394973754883
SF time: 0.013910055160522461
Abs time: 0.026507139205932617
Buyer init: 0.025794267654418945
SF time: 0.012515068054199219
Abs time: 0.024956464767456055
Cached runtime: 0.07865190505981445
{('A', 'B', 

In [ ]:
#first, create the csv file we'll want to write to
fh = open('zipf0_nocachetimes.csv', 'w+')
fieldnames = ['latency', 'cache_hit', 'use_aug']
csvwriter = csv.DictWriter(fh, fieldnames)
csvwriter.writeheader()

In [ ]:
#%%time
#...and compare against just doing original
#for this, we'll have to wipe memory completely before we can run this.
tot_ptime = 0.0
for i,b in enumerate(rand_b_seq):
  print("Looking at buyer {}".format(i))
  #start = time.time()
  #pr = cProfile.Profile()
  #pr.enable()
  _, new_time = find_optimal_plan(b, i, ground_truth)
  #pr.disable()
  #s = io.StringIO()
  #ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
  #ps.print_stats()
  #with open('buyer' + str(i) + '_norm_test.txt', 'w+') as f:
  #      f.write(s.getvalue())
  #end = time.time()
  tot_ptime += new_time
  print("time taken: {}".format(new_time))
  csvwriter.writerow({'latency' : new_time, 'cache_hit' : 0, 'use_aug' : 0})

print(tot_ptime)
fh.close()

Looking at buyer 0


/content/gdrive/My Drive/Win22/MLforFineTuning/factorizedexperiments/datamarket.py:384: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  join.drop('cov:c_y', 1, inplace=True)


seller243 ['seller243:seller243_att'] 0.9664136552660713
True
Number of dimensions tried: 300


/content/gdrive/My Drive/Win22/MLforFineTuning/factorizedexperiments/datamarket.py:329: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  join.drop('cov:c_x', 1, inplace=True)
/content/gdrive/My Drive/Win22/MLforFineTuning/factorizedexperiments/datamarket.py:330: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  join.drop('cov:c_y', 1, inplace=True)


seller287 ['seller243:seller243_att', 'seller287:seller287_att'] 1.000000000000001
True
Number of dimensions tried: 299
time taken: 6.362756013870239
Looking at buyer 1
seller252 ['seller252:seller252_att'] 0.8522439504333438
True
Number of dimensions tried: 300
seller80 ['seller252:seller252_att', 'seller80:seller80_att'] 0.9999999999999994
True
Number of dimensions tried: 299
time taken: 6.260364770889282
Looking at buyer 2
seller252 ['seller252:seller252_att'] 0.8686519830415865
True
Number of dimensions tried: 300
seller80 ['seller252:seller252_att', 'seller80:seller80_att'] 0.9999999999999993
True
Number of dimensions tried: 299
time taken: 6.198663711547852
Looking at buyer 3
seller252 ['seller252:seller252_att'] 0.9042200009876296
True
Number of dimensions tried: 300
seller80 ['seller252:seller252_att', 'seller80:seller80_att'] 0.9999999999999988
True
Number of dimensions tried: 299
time taken: 6.258225440979004
Looking at buyer 4
seller252 ['seller252:seller252_att'] 0.91019711